In [22]:
import pandas as pd
import numpy as np
import importlib
from leak_detect.base import * 

In [38]:
import leak_detect.base
importlib.reload(leak_detect.base)


<module 'leak_detect.base' from '/Users/abhaypawar/Documents/projects/leak_detection/leak-detect/leak_detect/base.py'>

In [39]:
data = pd.read_csv('data/daily_adjusted_AAPL.csv')
prediction_window = 5
data['target'] = data['open'].shift(-prediction_window)


In [60]:
#input_target_cols = ['open']
target_cols = ['target']
input_feature_cols = ['open', 'high', 'low', 'close']
output_feature_cols = ['return_5day', 'return_2day', 'open_1day_before_leaky']
#data_creation_func = create_features
#input_data = data.copy()


In [61]:
# Data creation function
def create_features(data_input):
    data = data_input.copy()
    # past 2 and 5 day return
    close_5days_before = data['target'].shift(5)
    close_2days_before = data['close'].shift(2)
    
    data['return_5day'] = (data['target'] - close_5days_before)/close_5days_before
    data['return_2day'] = (data['close'] - close_2days_before)/close_2days_before
    data['open_1day_before_leaky'] = data['open'].shift(-1)
    data['target'] = data['close']*3
    
    return data


## Horizontal leakge

In [62]:
detect_horizontal_leakage(create_features, data, target_cols, output_feature_cols, input_feature_cols)

Checking for leakage from target columns to feature columns...
By replacing target with NANs:
Oops horizontal leakage detected!! 
List of columns and their respective rows with leaky data:
return_5day : 5023


By adding imaginary component to target columns:
Oops horizontal leakage detected!! 
List of columns and their respective rows with leaky data:
return_5day : 5013


---------------------------------------------------------------------------
Checking for leakage from input feature columns to target columns...
By replacing input feature columns with NANs:
Oops horizontal leakage detected!! 
List of columns and their respective rows with leaky data:
target : 5033


By adding imaginary component to input feature columns:
Oops horizontal leakage detected!! 
List of columns and their respective rows with leaky data:
target : 5033




True

In [16]:
# things to do
# 1. horizontal leakage: input function, data, input_target_cols, output_target_cols, input_feature_cols, output_feature_cols
#   a. With nan. with complex
# 2. vertical leakage
# INPTUS: input function, data, columns, direction, check_row_number. OUTPUT: Number of rows till which 
# leakage happened

target_cols = ['target']
features_input_cols = ['open', 'close']

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient,target,return_5day,return_2day,close_1day_before
0,2020-06-19,354.635,356.56,345.15,349.72,349.7200,66118952,0.0,1.0,344.72,NaN,NaN,NaN
1,2020-06-18,351.410,353.45,349.22,351.73,351.7300,24205096,0.0,1.0,349.31,NaN,NaN,354.635
2,2020-06-17,355.150,355.40,351.09,351.59,351.5900,28601626,0.0,1.0,347.90,NaN,-0.008586,351.410
3,2020-06-16,351.460,353.20,344.72,352.08,352.0800,41357182,0.0,1.0,332.14,NaN,0.001907,355.150
4,2020-06-15,333.250,345.68,332.58,342.99,342.9900,34702230,0.0,1.0,330.25,NaN,-0.034239,351.460
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5028,2000-06-23,53.780,54.63,50.81,51.69,3.1960,3660100,0.0,1.0,NaN,-0.021208,-0.038862,52.500
5029,2000-06-22,55.750,57.63,53.56,53.75,3.3234,8352000,0.0,1.0,NaN,0.013004,0.023810,53.780
5030,2000-06-21,50.500,56.94,50.31,55.63,3.4396,4375000,0.0,2.0,NaN,0.043519,0.034399,55.750
5031,2000-06-20,98.500,103.94,98.37,101.25,3.1302,4476700,0.0,1.0,NaN,0.882670,0.816143,50.500


## Vertical leakge

In [ ]:
#input_target_cols = ['open']
input_feature_cols = ['open', 'high', 'low', 'close']
output_feature_cols = ['return_5day', 'return_2day', 'open_1day_before_leaky']

data_creation_func = create_features
input_data = data.copy()
